### Item-to-Item коллаборативная фильтрация

Рекомендации фильма на основе рейтинга фильмов

In [1]:
import pandas as pd
import numpy as np

In [7]:
movies = pd.read_csv('data/ml-25m/movies.csv')
ratings = pd.read_csv('data/ml-25m/ratings.csv')

In [3]:
movies.head(3)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [4]:
ratings.head(3)

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828


In [ ]:
#set_index on (присоединение индекса movieId по индексу movieId)
movies_with_ratings = movies.join(ratings.set_index('movieId'),on='movieId').reset_index(drop=True)

In [ ]:
movies_with_ratings.head(5)

In [7]:
#удалим NaN
movies_with_ratings.dropna(inplace=True)

In [8]:
movies_with_ratings.userId

0                2.0
1                3.0
2                4.0
3                5.0
4                8.0
              ...   
25003466    119571.0
25003467    115835.0
25003468      6964.0
25003469    119571.0
25003470    119571.0
Name: userId, Length: 25000095, dtype: float64

In [9]:
#уникальное число пользователей
num_users = movies_with_ratings.userId.unique().shape[0]
num_users

162541

In [34]:
movie_vector = {}
#группировка строк по заголовкам
for movie, group in movies_with_ratings.groupby('title'):
    #сперва вектор заполнен нулями, т.е если пользователь не смотрел фильм
    #то будет 0
    movie_vector[movie]=np.zeros(num_users)
    #заполняем вектор
    for i in range(len(group.userId.values)):
        user = group.userId.values[i]
        rating = group.rating.values[i]
        movie_vector[movie][int(user-1)]=rating

In [12]:
#импортируем расстояние
from scipy.spatial.distance import cosine, euclidean, hamming, correlation

In [64]:
type(movie_vector)

dict

In [66]:
#вывод вектора (матрица с фильмами и пользователями (0 если пользователь не смотрел фильм))
#вывод items() возвращает пары(ключ, значение)
list(movie_vector.items())[:5]

[('"BLOW THE NIGHT!" Let\'s Spend the Night Together (1983)',
  array([0., 0., 0., ..., 0., 0., 0.])),
 ('"Great Performances" Cats (1998)', array([0., 0., 0., ..., 0., 0., 0.])),
 ('#1 Cheerleader Camp (2010)', array([0., 0., 0., ..., 0., 0., 0.])),
 ('#Captured (2017)', array([0., 0., 0., ..., 0., 0., 0.])),
 ('#Female Pleasure (2018)', array([0., 0., 0., ..., 0., 0., 0.]))]

In [70]:
favorite_film = 'Toy Story (1995)'

titles = []
distances = []
#смотрим как коррелируют между собой фильмы (ищим похожий фильм)
#опираясь на рейтинг
for key in movie_vector.keys():
    if key == favorite_film:
        continue
    
    titles.append(key)
    distances.append(correlation(movie_vector[favorite_film], movie_vector[key]))
      
best_indexes = np.argsort(distances)[:10]
best_movies = [(titles[i], distances[i]) for i in best_indexes]

for movie in best_movies:
    print(movie)

('Toy Story 2 (1999)', 0.551006507194897)
('Willy Wonka & the Chocolate Factory (1971)', 0.6511756810314209)
('Back to the Future (1985)', 0.6562068418410386)
('Lion King, The (1994)', 0.656776432864756)
("Bug's Life, A (1998)", 0.6599777634641218)
('Aladdin (1992)', 0.6614160096207382)
('Independence Day (a.k.a. ID4) (1996)', 0.6617913359976116)
('Monsters, Inc. (2001)', 0.664391428963012)
('Mission: Impossible (1996)', 0.6797301234472074)
('Star Wars: Episode IV - A New Hope (1977)', 0.6807438359095375)


Чем меньше корреляция, тем больше соответсвие

Пользователям которым понравился фильм Toy Story (1995) нравятся ещё фильмы перечисленные выше

### User-Based коллаборативная фильтрация

In [2]:
from surprise import KNNWithMeans, KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import Reader

In [7]:
movies = pd.read_csv('data/ml-25m/movies.csv', nrows=10000)
ratings = pd.read_csv('data/ml-25m/ratings.csv', nrows=10000)
movies_with_ratings = movies.join(ratings.set_index('movieId'),on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [8]:
#фильмы которые смотрел пользователь 2
movies_with_ratings[movies_with_ratings.userId == 2.0].title.unique()[:20]

array(['Toy Story (1995)', "Mr. Holland's Opus (1995)",
       'Braveheart (1995)', 'Apollo 13 (1995)', 'Rob Roy (1995)',
       'French Kiss (1995)', 'Star Wars: Episode IV - A New Hope (1977)',
       'Little Women (1994)', 'Legends of the Fall (1994)',
       'Shawshank Redemption, The (1994)', 'Tommy Boy (1995)',
       'Clear and Present Danger (1994)', 'Forrest Gump (1994)',
       'Lion King, The (1994)', 'True Lies (1994)',
       'Fugitive, The (1993)', 'Jurassic Park (1993)',
       'Much Ado About Nothing (1993)', 'Rudy (1993)',
       "Schindler's List (1993)"], dtype=object)

In [9]:
dataset = pd.DataFrame({
    'user_id': movies_with_ratings.userId,
    'item_id': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [65]:
dataset.head()

,user_id,item_id,rating
0,2.0,Toy Story (1995),3.5
1,3.0,Toy Story (1995),4.0
2,4.0,Toy Story (1995),3.0
3,5.0,Toy Story (1995),4.0
4,8.0,Toy Story (1995),4.0


In [66]:
ratings.rating.min()

0.5

In [67]:
ratings.rating.max()

5.0

In [10]:
reader = Reader(rating_scale=(0.5, 5.0))#парсер (указали минимальную и максимальную оценку)
data = Dataset.load_from_df(dataset, reader)#для работы surprise
data #разряженная матрица (матрица пар пользователь - объект)
#если пользователь не взаимодействовал с объектом - пересечние = 0
#0 не хранит (не воспримает, для экономия оперативной памяти)

In [3]:
from surprise.model_selection import train_test_split

In [12]:
X_train, X_test = train_test_split(data, test_size=.15)

In [10]:
X_test[:10]

[(10.0, 'Tootsie (1982)', 3.0),
 (31.0, 'Enemy of the State (1998)', 2.5),
 (12.0, 'Deliverance (1972)', 4.0),
 (43.0, 'Hunt for Red October, The (1990)', 4.0),
 (43.0, 'Jerk, The (1979)', 3.0),
 (72.0, 'Jurassic Park (1993)', 4.0),
 (12.0, 'Godfather, The (1972)', 4.0),
 (70.0, 'Great Waldo Pepper, The (1975)', 3.0),
 (72.0, 'Airplane II: The Sequel (1982)', 5.0),
 (60.0, 'Say Anything... (1989)', 4.0)]

In [11]:
#pearson_baseline - корреляция
#user_based = True --> User Based рекомендательная система
#user_based = False --> item to item
knn = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
knn.fit(X_train)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [12]:
pred = knn.test(X_test)#test - оценивает значения

In [13]:
accuracy.rmse(pred, verbose=True)

RMSE: 0.9731


0.9730801977633079

Высокая ошибка, ошибаемся на 1 балл (завышаем или занижаем оценку)

In [14]:
#предсказываем, какой рейтинг пользователь 2 поставит фильму Pulp Fiction (1994)
#uid - user_id
#iid - item_id
result = knn.predict(uid=2, iid='Pulp Fiction (1994)')

In [15]:
#r_ui=None - Человек не смотрел фильм (нет оценки)
result

Prediction(uid=2, iid='Pulp Fiction (1994)', r_ui=None, est=4.226276675044499, details={'actual_k': 11, 'was_impossible': False})

In [16]:
print('Оценка пользователя 2 по мнению алгоритма'+' = '+ str(result.est))

Оценка пользователя 2 по мнению алгоритма = 4.226276675044499


### Item-Based коллаборативная фильтрация

Лучше работает, когда пользователей больше, чем объектов

In [13]:
knn = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': False})
knn.fit(X_train)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [18]:
pred = knn.test(X_test)

In [19]:
accuracy.rmse(pred, verbose=True)

RMSE: 0.9810


0.9810372198992078

Качество модели немного лучше, чем при использовании User-Based

In [20]:
result = knn.predict(uid=2, iid='Pulp Fiction (1994)')

In [21]:
result

Prediction(uid=2, iid='Pulp Fiction (1994)', r_ui=None, est=4.280928774794587, details={'actual_k': 50, 'was_impossible': False})

In [22]:
print('Оценка пользователя 2 по мнению алгоритма'+' = '+ str(result.est))

Оценка пользователя 2 по мнению алгоритма = 4.280928774794587


In [23]:
dataset.user_id.unique().shape[0], dataset.item_id.unique().shape[0]

(75, 2431)

Пользователей меньше, чем объектов, следовательно больше подходит User-Based. Качество лучше из-за маленького датасета

### Тоже самое на большом dataset'е

In [4]:
movies = pd.read_csv('data/ml-25m/movies.csv',nrows=500000)
ratings = pd.read_csv('data/ml-25m/ratings.csv',nrows=500000)
movies_with_ratings = movies.join(ratings.set_index('movieId'),on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [5]:
dataset = pd.DataFrame({
    'user_id': movies_with_ratings.userId,
    'item_id': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [6]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [7]:
X_train, X_test = train_test_split(data, test_size=.15)

#### User-Based

In [8]:
from surprise.model_selection import cross_validate

In [9]:
knn = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
pred = knn.fit(X_train).test(X_test)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [10]:
accuracy.rmse(pred, verbose=True)

RMSE: 0.8556


0.8556021997117592

In [21]:
cross_validate(knn, data, measures=['RMSE'], cv=5, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8607  0.8603  0.8550  0.8571  0.8643  0.8595  0.0032  
Fit time          15.77   15.16   15.29   15.29   15.37   15.37   0.21    
Test time         29.79   29.69   29.73   29.94   29.84   29.80   0.09    


{'test_rmse': array([0.86067244, 0.86034751, 0.85498996, 0.8571301 , 0.86434167]),
 'fit_time': (15.768654823303223,
  15.160407066345215,
  15.286414861679077,
  15.28511905670166,
  15.366268634796143),
 'test_time': (29.7948956489563,
  29.68511986732483,
  29.729847192764282,
  29.936346530914307,
  29.83515167236328)}

#### Item-Based

In [11]:
knn2 = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': False})
pred2 = knn2.fit(X_train).test(X_test)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [12]:
accuracy.rmse(pred2, verbose=True)

RMSE: 0.8354


0.8354191191585345

In [13]:
cross_validate(knn2, data, measures=['RMSE'], cv=5, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8403  0.8411  0.8468  0.8457  0.8442  0.8436  0.0025  
Fit time          23.76   23.73   23.84   23.79   23.74   23.77   0.04    
Test time         23.51   23.85   23.68   23.62   23.65   23.66   0.11    


{'test_rmse': array([0.8402748 , 0.84108654, 0.84681525, 0.84570415, 0.84420956]),
 'fit_time': (23.756664991378784,
  23.73120427131653,
  23.835771322250366,
  23.79173445701599,
  23.741585969924927),
 'test_time': (23.509082078933716,
  23.848726749420166,
  23.682634353637695,
  23.617732286453247,
  23.654983043670654)}